In [9]:
import numpy as np
import pandas as pd

In [10]:
# Chargement des DataSets
fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')
heart_df = pd.read_csv('../../../Data/Classification/Indicators of Heart Disease/heart_2022_no_nans_clean.csv')

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15455/302461350.py:2: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')


In [11]:
def target_accuracy(df, pollution_levels, target_column, noise_factor=1):
    df_dict = {}

    for pollution_percentage in pollution_levels:
        df_polluted = df.copy()

        # Vérifier si la colonne cible est catégorique ou numérique
        if df_polluted[target_column].dtype == 'object' or pd.api.types.is_categorical_dtype(df_polluted[target_column]):
            # Pollution des données catégoriques
            unique_values = df_polluted[target_column].unique()

            # Sélectionner un sous-ensemble de lignes à polluer
            mask = np.random.rand(len(df_polluted)) < pollution_percentage

            # Pour chaque ligne sélectionnée, changer la valeur par une autre valeur aléatoire différente
            df_polluted.loc[mask, target_column] = df_polluted.loc[mask, target_column].apply(
                lambda x: np.random.choice(unique_values[unique_values != x])
            )

        elif np.issubdtype(df_polluted[target_column].dtype, np.number):
            # Pollution des données numériques
            mask = np.random.rand(len(df_polluted)) < pollution_percentage

            # Vérifier si la colonne ne contient que des valeurs entières (même si elles sont de type float)
            is_integer_column = df_polluted[target_column].dropna().apply(lambda x: x == int(x)).all()

            if is_integer_column:
                # Ajouter du bruit gaussien aux valeurs sélectionnées
                noise = np.random.normal(0, df_polluted[target_column].std() * noise_factor, size=len(df_polluted[mask]))
                df_polluted.loc[mask, target_column] += noise

                # Arrondir à l'entier le plus proche
                df_polluted[target_column] = df_polluted[target_column].round(0)
            else:
                # Récupérer le nombre de décimales de la première valeur non nulle pour garder la précision
                precision = df_polluted[target_column].dropna().apply(
                    lambda x: len(str(x).split('.')[1]) if '.' in str(x) else 0
                ).max()

                # Ajouter du bruit gaussien aux valeurs sélectionnées
                noise = np.random.normal(0, df_polluted[target_column].std() * noise_factor, size=len(df_polluted[mask]))
                df_polluted.loc[mask, target_column] += noise

                # Arrondir à la précision originale
                df_polluted[target_column] = df_polluted[target_column].round(precision)

        else:
            raise ValueError("Le type de la colonne cible n'est ni catégorique ni numérique.")

        # Ajouter le DataFrame pollué au dictionnaire avec le pourcentage de pollution comme clé
        df_dict[f"{int(pollution_percentage * 100)}%"] = df_polluted

    return df_dict

In [12]:
# Liste des pourcentages de valeurs manquantes à introduire
pollution_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# Génération des DataFrames avec valeurs manquantes
fps_df_with_target_accuracy = target_accuracy(fps_df, pollution_levels,'FPS')
heart_df_with_target_accuracy = target_accuracy(heart_df, pollution_levels,'HadHeartAttack')

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15455/941028117.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df_polluted[target_column].dtype == 'object' or pd.api.types.is_categorical_dtype(df_polluted[target_column]):
/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15455/941028117.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df_polluted[target_column].dtype == 'object' or pd.api.types.is_categorical_dtype(df_polluted[target_column]):
/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15455/941028117.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df_polluted[target_column].dtype == 'object' or pd.api.types.is_categorical_dty

In [13]:
# Enregistrement des DataFrames générés
for key, value in fps_df_with_target_accuracy.items():
    value.to_csv(f"../../../Data/Regression/fps in video games/Target Accuracy/fps_{key}.csv", index=False)

for key, value in heart_df_with_target_accuracy.items():
    value.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Target Accuracy/heart_{key}.csv", index=False)